# 异常值检测

## 背景

由于在临床数据中，患者的检测值往往本身就偏离正常范围（如糖尿病患者的血糖、高血压患者的血压），传统的基于健康人群参考区间的异常值检测方法可能不适用，因此异常值处理十分棘手。这里提出两种异常值检测方法。

## 检测方法

孤立森林(isolation Forest)算法，2008年由刘飞、周志华等提出，算法不借助类似距离、密度等指标去描述样本与其他样本的差异，而是直接去刻画所谓的疏离程度(isolation)，因此该算法简单、高效，在工业界应用较多。
孤立森林算法适合高维稀疏数据，对分布无假设，因此适用于当前数据。

In [ ]:
import pandas as pd
import numpy as np

# Load data from a pickle file
try:
    data : pd = pd.read_pickle('../data/data_remove_dup.pkl')
except:
    print("未在路径中找到目标数据表，先运行其他脚本")
    raise

data

In [ ]:
from sklearn.ensemble import IsolationForest

# 初始化模型（调整contamination参数控制异常比例）
clf = IsolationForest(
    n_estimators=100,
    contamination=0.02,  # 假设2%异常
    random_state=42
)

testData = data.iloc[:, 8:]

# 训练并预测
data['anomaly_score'] = clf.fit_predict(testData.fillna(testData.median()))

data['is_anomaly'] = np.where(data['anomaly_score'] == -1, True, False)
anomalies = data[data['is_anomaly']]

anomalies

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
# 计算异常样本的偏离值（相对于全局中位数）
median_values = data.iloc[:, 8:].median()
anomalies_deviation = anomalies.iloc[:, 8:] - median_values

matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体
matplotlib.rcParams['axes.unicode_minus'] = False

# 绘制热力图
plt.figure(figsize=(12, 8))
sns.heatmap(
    anomalies_deviation.T,
    cmap='coolwarm',
    annot=False,
    fmt=".2f",
)
plt.title("异常样本各指标偏离中位数的程度")
plt.xlabel("样本编号")
plt.ylabel("检测指标")
plt.xticks(rotation=90)
plt.show()